In [163]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), "nytimesarticle-0.1.0"))
from lxml import html
from lxml import etree
from bs4 import BeautifulSoup
import nytimesarticle
import pandas as pd
import requests
import re
import datetime
import json

In [21]:
with open('nyt_keys.json') as keys:
    nyt_creds = json.loads(keys)

#api = nytimesarticle.search()
api = nytimesarticle.articleAPI(nyt_creds['NYT_ARTICLE'])
start_dt = pd.to_datetime('20161119').date()
end_dt = pd.to_datetime('20161126').date()
num_days = (end_dt - start_dt).days
start_dt = str(start_dt.year)+str(start_dt.month).zfill(2)+str(start_dt.day).zfill(2)
end_dt = str(end_dt.year)+str(end_dt.month).zfill(2)+str(end_dt.day).zfill(2)

In [108]:
print start_dt
print end_dt

20161119
20161126


In [23]:
results = api.search(q = 'brexit', fq = {'source':['Reuters','AP', 'The New York Times'],\
                                          'news_desk':["Foreign","Business","Financial","Market Place","World"],
                                          'document_type':"article"}, begin_date = start_dt, end_date = end_dt, \
                      sort= "newest", \
                      fl = "web_url,snippet,lead_paragraph,abstract,source,headline,keywords,pub_date,document_type,news_desk,type_of_material", \
                      facet_field = "source,section_name,document_type")

In [90]:
num_items = len(results['response']['docs'])
corpus = []
for i in xrange(num_items):
    r = requests.get(results['response']['docs'][i]['web_url'])
    tree = html.fromstring(r.content)
    contents = tree.xpath('//p[@class="story-body-text story-content"]/text()')
    article = []
    for t in contents: article.append(re.sub(r'[^\w]', ' ', t))
    corpus.append(''.join(article))
print corpus

http://www.nytimes.com/2016/11/23/world/europe/uk-philip-hammond-autumn-statement-brexit.html
2016-11-23T09:54:45+0000
[u'LONDON     After years of tough spending curbs     s government on Wednesday cast aside the language of austerity as it acknowledged the high economic cost of withdrawal from the European Union  and tried to placate struggling working class families whose incomes have stagnated A pledge by the previous chancellor of the Exchequer  or finance minister  George Osborne  to balance the budget by 2020 has been shelved  The current chancellor  Philip Hammond  is replacing it with a vaguer ambition to do so as soon as practical after 2020 Help for workers whose low earnings are supplemented by welfare payments  an increase in the minimum wage and new infrastructure spending were features of the first Autumn Statement  or spending plan  from Mr  Hammond  who became chancellor in July Changes were centered on a section of society identified by Prime Minister Theresa May as t

In [502]:
def get_NYT(start, end):
    NYT_ARTICLE = 'deffb8ca21824660b6e431b160550918'
    NYT_SEMANTIC = 'deffb8ca21824660b6e431b160550918'
    #api = nytimesarticle.search()
    api = nytimesarticle.articleAPI(NYT_ARTICLE)
    start_dt = pd.to_datetime(start).date()
    end_dt = pd.to_datetime(end).date()
    num_days = (end_dt - start_dt).days
    articles = []
    for i in xrange(num_days+1):
        dt = start_dt + datetime.timedelta(i)
        dt_start = str(dt.year)+str(dt.month).zfill(2)+str(dt.day).zfill(2)
        dt_end = str(dt.year)+str(dt.month).zfill(2)+str(dt.day+1).zfill(2)
        #print dt_start
        #print dt_end
        result = api.search(q = 'brexit', fq = {'source':['Reuters','AP', 'The New York Times'],\
                                                  'news_desk':["Foreign","Business","Financial","Market Place","World"],
                                                  'document_type':"article"}, begin_date = dt_start, end_date = dt_start, \
                              sort= "newest", \
                              fl = "web_url,snippet,lead_paragraph,abstract,source,headline,keywords,pub_date,document_type,news_desk,type_of_material", \
                              facet_field = "source,section_name,document_type")
        for r in result:
            if r == unicode('response'):
                for k in result[r].keys():
                    if k == unicode('docs'):
                        for doc in result[r][k]:
                            articles.append(doc)
        #print len(articles)
                            
    dates = []
    corpus = []
    print len(articles)
    for art in articles:
        r = requests.get(art[unicode('web_url')])
        #print r
        tree = html.fromstring(r.content)
        contents = tree.xpath('//p[@class="story-body-text story-content"]/text()')
        body = []
        for t in contents: 
            if t.strip <> '': body.append(re.sub(r'[^\w]', ' ', t))
        corpus.append(''.join(body))
        dates.append(art[unicode('pub_date')][0:10])
    return dates, corpus

In [498]:
pd_corpus_2 = pd.DataFrame(corpus,index=pd.to_datetime(dts))

In [475]:
pd_corpus.to_csv('NYTimes_20160623-20161127.csv')

In [ ]:
if __name__ == '__main__':
	get_fx(sys.argv[1],sys.argv[2],sys.argv[3])